# Getting started

The `pivmetalib` implements many (but not all) classes for various ontologies and vocabularies used in the ontology ["pivmeta"](https://matthiasprobst.github.io/pivmeta/).

By means of `pivmetalib` you are able to instantiate objects of the ontology `pivmeta` (and the related ones).

Let's find out, what that means for the description of a Person (A person can be described by the PROV namespace: https://www.w3.org/ns/prov)

In [1]:
from pivmetalib import prov

In [2]:
creator = prov.Person(
    lastName='Okamoto',
    mbox="okamoto@tokai.t.u-tokyo.ac.jp"
)
creator

Person(mbox=okamoto@tokai.t.u-tokyo.ac.jp, lastName=Okamoto)

Most importantly, dumping the data to a JSON-LD string is most practical and useful for use in data description. For this call `model_dump_jsonld()`:

In [3]:
print(creator.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "local": "http://example.org/",
        "prov": "http://www.w3.org/ns/prov#",
        "foaf": "http://xmlns.com/foaf/0.1/"
    },
    "@type": "prov:Person",
    "foaf:mbox": "okamoto@tokai.t.u-tokyo.ac.jp",
    "foaf:lastName": "Okamoto",
    "@id": "local:e8c8047a-c467-4539-a640-2fb4e0d31f37"
}


**What's behind the class:** Each class is inherited from the `ontolutils.Thing`, which again is a `pydantic.BaseModel` subclass. The package [`pydantic`](https://docs.pydantic.dev/latest/) allows validating the attributes of a class. This means, that passing wrong types to the person, will lead to an error:

In [4]:
import pydantic

In [5]:
try:
    prov.Person(lastName='John', mbox='email.com')
except pydantic.ValidationError as e:
    print(e)

1 validation error for Person
mbox
  value is not a valid email address: The email address is not valid. It must have exactly one @-sign. [type=value_error, input_value='email.com', input_type=str]


**Extra fields**

Generally, it is possible to add any additional fields, like age, for example. The field "age" will wil appear in the JSON-LD string, however without a *prefix*:

In [6]:
creator = prov.Person(
    firtName='John',
    lastName='Doe',
    age=34
)
print(creator.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "local": "http://example.org/",
        "prov": "http://www.w3.org/ns/prov#",
        "foaf": "http://xmlns.com/foaf/0.1/"
    },
    "@type": "prov:Person",
    "foaf:lastName": "Doe",
    "firtName": "John",
    "age": "34",
    "@id": "local:336cbdd4-5946-445c-a710-ce0f170920db"
}


To change this, we can update the missing URI at all times:

In [13]:
creator.urirefs['age'] = 'foaf:age'

In [14]:
print(creator.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "local": "http://example.org/",
        "prov": "http://www.w3.org/ns/prov#",
        "foaf": "http://xmlns.com/foaf/0.1/"
    },
    "@type": "prov:Person",
    "foaf:lastName": "Doe",
    "firtName": "John",
    "foaf:age": "34",
    "@id": "local:483ed129-b20e-4b8d-996d-72cf6b0db57b"
}


## Working with the classes

There are two importing things, you can do with the classes:
- **Dumping**: This means exporting the content to various formats (dictionary, json-ld strings, ...)
- **querying**: Finding data of a class in a JSON-LD file

### Dumping:
The most basic form of "dumping" is exporting the class instance to a dictionary:

In [15]:
creator.model_dump()

{'id': None,
 'label': None,
 'mbox': None,
 'firstName': None,
 'lastName': 'Doe',
 'hadRole': None,
 'wasRoleIn': None,
 'affiliation': None,
 'firtName': 'John',
 'age': 34}

The `pivmetalib` adds also `dump_jsonld` to the classes and allows to create a JSON-LD string, which can be saved to a file, too:

In [16]:
print(creator.model_dump_jsonld())

{
    "@context": {
        "owl": "http://www.w3.org/2002/07/owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "local": "http://example.org/",
        "prov": "http://www.w3.org/ns/prov#",
        "foaf": "http://xmlns.com/foaf/0.1/"
    },
    "@type": "prov:Person",
    "foaf:lastName": "Doe",
    "firtName": "John",
    "foaf:age": "34",
    "@id": "local:629b6d5a-c1f5-4b4e-a402-94310e8f6f62"
}


**Save JSON-LD file**

In [17]:
import pivmetalib

In [18]:
# save to file:
with open('john.jsonld', 'w') as f:
    creator.model_dump_jsonld(context={"@import": pivmetalib.CONTEXT})

Let's create another person and save both to the file. For this we will need to "merge" both JSON-LD strings. There is a helper function for this in the `utils` module: 

In [19]:
other_person = prov.Person(first_name='Lisa')

In [20]:
from pivmetalib.jsonld import merge

In [21]:
# save to file:
with open('creator.jsonld', 'w') as f:
    f.write(merge([creator.model_dump_jsonld(context={"@import": pivmetalib.CONTEXT}),
                   other_person.model_dump_jsonld(context={"@import": pivmetalib.CONTEXT})]))

### Querying

We can identify data within a JSON-LD file by calling `query` and providing the class we want to find. The method applies a SPARQL query ad returns an instance of the provided object:

In [22]:
from ontolutils import query

In [27]:
persons = query(cls=prov.Person, source='creator.jsonld', context={"@import": pivmetalib.CONTEXT})
persons

[Person(id=http://example.org/c9013111-000c-4c5e-8c4d-f4fb743a8779, lastName=Doe, firtName=John, age=34),
 Person(id=http://example.org/d4f6000d-bcf1-4795-a724-54987a59956d, first_name=Lisa)]

Likewise, you can call `from_jsonld` from the Person object:

In [28]:
persons = prov.Person.from_jsonld(source='creator.jsonld', context={"@import": pivmetalib.CONTEXT})
persons

[Person(id=http://example.org/c9013111-000c-4c5e-8c4d-f4fb743a8779, lastName=Doe, firtName=John, age=34),
 Person(id=http://example.org/d4f6000d-bcf1-4795-a724-54987a59956d, first_name=Lisa)]

As expected, we find both persons:

In [29]:
import rdflib

In [30]:
g = rdflib.Graph().parse(source='creator.jsonld', context=pivmetalib.CONTEXT)

In [31]:
for s,p,o in g:
    print(s, o)

http://example.org/c9013111-000c-4c5e-8c4d-f4fb743a8779 http://www.w3.org/ns/prov#Person
http://example.org/c9013111-000c-4c5e-8c4d-f4fb743a8779 Doe
http://example.org/d4f6000d-bcf1-4795-a724-54987a59956d http://www.w3.org/ns/prov#Person
http://example.org/d4f6000d-bcf1-4795-a724-54987a59956d Lisa
http://example.org/c9013111-000c-4c5e-8c4d-f4fb743a8779 34
http://example.org/c9013111-000c-4c5e-8c4d-f4fb743a8779 John


In [32]:
qs = """PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX local: <http://example.com/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT *
WHERE {
    ?id a prov:Person .
    ?id ?p ?o .
}"""

In [33]:
for r in g.query(qs):
    print(r)

(rdflib.term.URIRef('http://www.w3.org/ns/prov#Person'), rdflib.term.URIRef('http://example.org/c9013111-000c-4c5e-8c4d-f4fb743a8779'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'))
(rdflib.term.Literal('Doe'), rdflib.term.URIRef('http://example.org/c9013111-000c-4c5e-8c4d-f4fb743a8779'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/lastName'))
(rdflib.term.Literal('John'), rdflib.term.URIRef('http://example.org/c9013111-000c-4c5e-8c4d-f4fb743a8779'), rdflib.term.URIRef('https://matthiasprobst.github.io/pivmeta#firtName'))
(rdflib.term.Literal('34'), rdflib.term.URIRef('http://example.org/c9013111-000c-4c5e-8c4d-f4fb743a8779'), rdflib.term.URIRef('http://xmlns.com/foaf/0.1/age'))
(rdflib.term.URIRef('http://www.w3.org/ns/prov#Person'), rdflib.term.URIRef('http://example.org/d4f6000d-bcf1-4795-a724-54987a59956d'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'))
(rdflib.term.Literal('Lisa'), rdflib.term.URIRef('http://example.org/d4f6000d-

In [34]:
for person in persons:
    print(person.model_dump(exclude_none=True))

{'id': 'http://example.org/c9013111-000c-4c5e-8c4d-f4fb743a8779', 'lastName': 'Doe', 'firtName': 'John', 'age': '34'}
{'id': 'http://example.org/d4f6000d-bcf1-4795-a724-54987a59956d', 'first_name': 'Lisa'}
